In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import pickle
import pprintpp
#import baseballAux as aux

GAME_LOGS_FILE = "./data/extracted/baseball/game_logs.csv"
DICT_SPECS_FILE = "./data/extracted/baseball/keysDict.pickle"


###############################################################################
# Loading the dataset
gl = pd.read_csv(GAME_LOGS_FILE)
gl.info(memory_usage='deep')
gl.head()

/Users/roberto/anaconda3/envs/NuevoCADI/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (12,13,14,15,19,20,81,83,85,87,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(6), object(78)
memory usage: 861.6 MB


,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,18710508,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,18710509,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


In [3]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj, pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else:  # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2  # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)


def stringMemoryComparison(before, after):
    string = "B: " + str(mem_usage(before)) + "\n" + "A: " + str(mem_usage(after))
    return string

In [4]:
# Exploring memory usage
for dtype in ['float', 'int', 'object']:
    selected_dtype = gl.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print(
        "Average memory usage for {} columns: {:03.2f}MB".format(
            dtype, mean_usage_mb
        )
    )


Average memory usage for float columns: 1.29MB
Average memory usage for int columns: 1.12MB
Average memory usage for object columns: 9.53MB


In [5]:
# Exploring types' memory footprint
int_types = ["uint8", "int8", "int16"]
for it in int_types:
    print(np.iinfo(it))
float_types = ["float16", "float32", "float64"]
for it in float_types:
    print(np.finfo(it))



Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------

Machine parameters for float16
---------------------------------------------------------------
precision =   3   resolution = 1.00040e-03
machep =    -10   eps =        9.76562e-04
negep =     -11   epsneg =     4.88281e-04
minexp =    -14   tiny =       6.10352e-05
maxexp =     16   max =        6.55040e+04
nexp =        5   min =        -max
---------------------------------------------------------------

Machine parameters for float32
---------------------

In [7]:
gl_int = gl.select_dtypes(include=['int'])
converted_int = gl_int.apply(pd.to_numeric, downcast='unsigned')
print(stringMemoryComparison(gl_int, converted_int))


B: 7.87 MB
A: 1.48 MB


In [9]:
gl_float = gl.select_dtypes(include=['float'])
converted_float = gl_float.apply(pd.to_numeric, downcast='float')
print(stringMemoryComparison(gl_float, converted_float))


B: 100.99 MB
A: 50.49 MB


In [11]:

optimized_gl = gl.copy()
optimized_gl[converted_int.columns] = converted_int
optimized_gl[converted_float.columns] = converted_float
print(stringMemoryComparison(gl, optimized_gl))


B: 861.57 MB
A: 804.69 MB


In [13]:
# Optimizing using categoricals
gl_obj = gl.select_dtypes(include=['object']).copy()
gl_obj.describe()

dow = gl_obj.day_of_week
print(dow.head())

dow_cat = dow.astype('category')
print(dow_cat.head())

dow_cat.head().cat.codes
print(stringMemoryComparison(dow, dow_cat))

0    Thu
1    Fri
2    Sat
3    Mon
4    Tue
Name: day_of_week, dtype: object
0    Thu
1    Fri
2    Sat
3    Mon
4    Tue
Name: day_of_week, dtype: category
Categories (7, object): [Fri, Mon, Sat, Sun, Thu, Tue, Wed]
B: 9.84 MB
A: 0.16 MB


In [15]:
# Optimizing using categoricals throughout the dataframe
converted_obj = pd.DataFrame()
for col in gl_obj.columns:
    num_unique_values = len(gl_obj[col].unique())
    num_total_values = len(gl_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:, col] = gl_obj[col].astype('category')
    else:
        converted_obj.loc[:, col] = gl_obj[col]
print(stringMemoryComparison(gl_obj, converted_obj))

converted_obj.head()
optimized_gl[converted_obj.columns] = converted_obj
print(stringMemoryComparison(gl, optimized_gl))


B: 752.72 MB
A: 51.67 MB
B: 861.57 MB
A: 103.64 MB


In [17]:
# Optimizing the date
date = optimized_gl.date
optimized_gl['date'] = pd.to_datetime(date, format='%Y%m%d')
print(stringMemoryComparison(gl, optimized_gl))



B: 861.57 MB
A: 104.29 MB


In [18]:
###############################################################################
# Dump the dictionary of dataspecs for use in the next exercise
dtypes = optimized_gl.drop('date', axis=1).dtypes
dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]
column_types = dict(zip(dtypes_col, dtypes_type))
preview = first2pairs = {
    key: value for key, value in list(column_types.items())[:10]
}
pp = pprintpp.PrettyPrinter(indent=4)
pp.pprint(preview)
with open(DICT_SPECS_FILE, 'wb') as handle:
    pickle.dump(column_types, handle, protocol=pickle.HIGHEST_PROTOCOL)


{
    'day_of_week': 'category',
    'h_game_number': 'uint8',
    'h_league': 'category',
    'h_name': 'category',
    'h_score': 'uint8',
    'number_of_game': 'uint8',
    'v_game_number': 'uint8',
    'v_league': 'category',
    'v_name': 'category',
    'v_score': 'uint8',
}
